# Laboratorio 4: Agents y Tools

## Introducción

En este laboratorio práctico sobre Agentes y Tools explorarás cómo construir agentes inteligentes utilizando las librerías LangChain y LangGraph.

A través de una serie de ejercicios prácticos aprenderás a:

- Crear tu primer Agente ReAct.
- Integrar múltiples tools en un agente.
- Construir un chatbot con LangGraph.
- Agregar herramientas externas como Wikipedia y APIs de clima.
- Incorporar una base de conocimiento utilizando un índice de Pinecone.
- Implementar condiciones personalizadas para la selección de tools.

Este laboratorio está diseñado para ser un espacio de práctica donde podrás experimentar y aplicar los conceptos de agentes y tools en un entorno interactivo.

In [ ]:
%pip install -U \
  langchain==0.3.7 \
  langchain-core==0.3.18 \
  langchain-openai==0.2.3 \
  langchain-community==0.3.7 \
  langchain-pinecone==0.2.0 \
  langchain-huggingface==0.1.2 \
  langgraph==0.2.19 \
  sentence-transformers \
  pinecone-client \
  pydantic==2.9.2 \
  pypdf

## Parte 1: Creando tu primer ReAct Agent

Crea un agente ReAct que use un modelo de lenguaje (`ChatOpenAI`) y una tool para contar cuántas letras "r" hay en una palabra.

El agente debe recibir una pregunta del usuario, invocar la herramienta y responder con el resultado.

In [ ]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from google.colab import userdata

# ✅ Inicializar el modelo de lenguaje gpt-4o-mini usando ChatOpenAI
model = ChatOpenAI(api_key=userdata.get('OPENAI_API_TOKEN'), model="gpt-4o-mini")

# ✅ Definir una tool que cuente las 'r' en una palabra
@tool
def count_r_in_word(word: str) -> int:
    """Count how many 'r' letters are in the given word."""
    return word.lower().count('r')

# ✅ Crear el agente con create_react_agent
app = create_react_agent(model=model, tools=[count_r_in_word])

# Pregunta del usuario
query = "How many r's are in the word 'Terrarium'?"

# ✅ Invocar el agente y mostrar la respuesta final
response = app.invoke({"messages": [("user", query)]})
print(response['messages'][-1].content)

## Parte 2: Agregando múltiples tools

Amplía el agente para incluir una segunda herramienta que calcule el área de un rectángulo.

Luego, haz una consulta que combine ambas herramientas en una sola interacción.

In [ ]:
# ✅ Define una tool que calcule el área de un rectángulo
@tool
def calculate_rectangle_area(length: float, width: float) -> float:
    """Calculate the area of a rectangle given its length and width."""
    return length * width

# ✅ Crear el agente con ambas tools
app = create_react_agent(model=model, tools=[count_r_in_word, calculate_rectangle_area])

query = "What is the area of a rectangle with length 5 and width 11? and how many r's are in 'Race'?"

# ✅ Invocar y mostrar la respuesta
response = app.invoke({"messages": [("user", query)]})
print(response['messages'][-1].content)

## Parte 3: Construyendo un chatbot con LangGraph

Crea un `StateGraph` básico con un solo nodo chatbot que responda preguntas directamente usando el modelo LLM.

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from google.colab import userdata

# ✅ Inicializa el modelo LLM
llm = ChatOpenAI(api_key=userdata.get('OPENAI_API_TOKEN'), model="gpt-4o-mini")

# ✅ Crea el grafo y define el nodo chatbot
graph_builder = StateGraph(MessagesState)

def chatbot(state: MessagesState):
    return {"messages": [llm.invoke(state["messages"])]}

# ✅ Agrega nodos y bordes al grafo
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

# ✅ Invoca el grafo con una consulta
response = graph.invoke({"messages": [("human", "Who is Ada Lovelace?")]})
print(response["messages"][-1].content)

## Parte 4: Agregando WikipediaTool y usando stream_tool_responses

Integra la tool de Wikipedia al chatbot y usa la función `stream_tool_responses` para observar cómo el agente decide cuándo invocar la herramienta y cómo se entregan las respuestas en streaming.

In [ ]:
%pip install wikipedia

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END

# ✅ Inicializa el wrapper de Wikipedia
api_wrapper = WikipediaAPIWrapper()
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

tools = [wikipedia_tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# ✅ Crea el grafo y define el nodo chatbot
graph_builder = StateGraph(MessagesState)
graph_builder.add_node("chatbot", chatbot)

# ✅ Crea el nodo de herramientas y define las transiciones
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

def stream_tool_responses(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for item in event.values():
            print("Agent:", item["messages"])

query = "Who is Alan Turing?"
stream_tool_responses(query)


## Parte 5: Agregando una Tool de API externa

Agrega una tool que obtenga el clima actual de una ciudad usando la API de Open-Meteo.

In [ ]:
import requests
from langchain_core.tools import tool

# ✅ Completar la función para obtener el clima actual
@tool
def weather_tool(city: str) -> str:
    """
    Retrieve current weather for a city using Open-Meteo.
    """
    # ✅ Paso 1: Geocodificación para obtener latitud y longitud
    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}"
    geo_resp = requests.get(geo_url).json()
    if "results" not in geo_resp or len(geo_resp["results"]) == 0:
        return f"Could not find location for {city}."

    lat = geo_resp["results"][0]["latitude"]
    lon = geo_resp["results"][0]["longitude"]

    # ✅ Paso 2: Obtener datos del clima actual
    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    weather_resp = requests.get(weather_url).json()

    if "current_weather" not in weather_resp:
        return f"Could not retrieve weather data for {city}."

    temp = weather_resp["current_weather"]["temperature"]
    wind = weather_resp["current_weather"]["windspeed"]
    return f"The current temperature in {city} is {temp}°C with a wind speed of {wind} km/h."

# ✅ Agrega la nueva tool al grafo
tools = [wikipedia_tool, weather_tool]
llm_with_tools = llm.bind_tools(tools)

# ✅ Volver a crear y compilar el grafo
graph_builder = StateGraph(MessagesState)
graph_builder.add_node("chatbot", lambda state: {"messages": [llm_with_tools.invoke(state["messages"])]})

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

# 🧪 Espacio para hacer pruebas con consultas
query = "What's the weather like in Montevideo and who was Nikola Tesla?"
response = graph.invoke({"messages": [("user", query)]})
print(response["messages"][-1].content)

## Parte 6: Agregando una base de conocimiento

Agrega una tool que consulte un índice de Pinecone con embeddings de un documento PDF.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
import os

# --- Configurar credenciales para HuggingFace y Pinecone ---
os.environ["HUGGINGFACE_API_KEY"] = userdata.get("HUGGING_API_KEY")
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")

# --- Cargar un archivo PDF desde el entorno de Colab ---
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
print(f"📄 Archivo cargado: {pdf_path}")

# --- Leer el contenido del PDF y dividirlo en fragmentos manejables ---
loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
chunks = text_splitter.split_documents(documents)
print(f"✅ Documento dividido en {len(chunks)} fragmentos")

# --- Crear embeddings y conectarse a Pinecone ---
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index_name = "docs-index"

# --- Crear el índice si aún no existe ---
if index_name not in [idx["name"] for idx in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensión del modelo de embeddings usado
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"✅ Índice '{index_name}' creado correctamente.")
else:
    print(f"ℹ️ El índice '{index_name}' ya existe.")

# --- Generar y subir los embeddings del PDF al índice Pinecone ---
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore.from_documents(chunks, embeddings, index_name=index_name)
print("✅ Embeddings generados y cargados en Pinecone correctamente.")

In [ ]:
# ==========================================
# Conexión y uso de la base de conocimiento (Pinecone)
# ==========================================

# Se conecta a un índice existente en Pinecone que ya contiene los embeddings del PDF
vector_store = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)
print(f"🔗 Conectado al índice existente '{index_name}' en Pinecone.")


# --- Tool de consulta: accede al índice para responder preguntas del PDF ---
@tool
def db_knowledge(query: str) -> str:
    """Busca fragmentos relevantes en el índice Pinecone y devuelve su contenido."""
    retriever = vector_store.as_retriever(search_kwargs={"k": 4})
    docs = retriever.get_relevant_documents(query)
    if not docs:
        return "No relevant knowledge found."
    return "\n\n---\n\n".join([doc.page_content for doc in docs])


# --- Construcción del grafo LangGraph ---
# El grafo define cómo el agente decide entre responder o usar la tool.
tools = [db_knowledge]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: MessagesState):
    # Instrucción para que el modelo use la tool si la pregunta está relacionada con el PDF
    system_prompt = (
        "You have access to a Pinecone vector store built from a PDF. "
        "If the question can be answered using that PDF, call the 'db_knowledge' tool first."
    )
    messages = [("system", system_prompt)] + state["messages"]
    return {"messages": [llm_with_tools.invoke(messages)]}

# Grafo con dos nodos: el chatbot y las tools
graph_builder = StateGraph(MessagesState)
graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# Define el flujo: si el modelo llama una tool → ir a 'tools'; si no → terminar (END)
graph_builder.add_conditional_edges("chatbot", tools_condition, {"tools": "tools", END: END})
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()
print("✅ Grafo compilado correctamente (usando índice existente).")


# --- Ejemplo de consulta ---
# Se envía una pregunta al agente, que decidirá si usar la base de conocimiento.
query = "Summarize the main ideas discussed in the uploaded PDF document."
response = graph.invoke({"messages": [("user", query)]})

print("\n🤖 Respuesta del agente:\n")
print(response["messages"][-1].content)

## Parte 7: Condiciones personalizadas para decisión de uso de tools

Crea un agente que decida entre las diferentes tools según el contenido del mensaje del usuario:

- Si el mensaje menciona "clima" → invocar `weather_tool`.
- Si menciona "wiki" → invocar `wikipedia_tool`.
- Si menciona "documento" → invocar `db_knowledge`.
- Para cualquier otro mensaje → invocar al LLM y terminar el flujo sin llamar a ninguna tool.

**Nota:** Tener en cuenta que cada tool debe ser creada en un nodo independiente.

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display
import re

def call_wikipedia_tool(query: str) -> str:
    """Llama a la tool de Wikipedia que ya creaste."""
    # WikipediaQueryRun expone `.run(texto)`
    return wikipedia_tool.run(query)


def extract_city_from_text(text: str) -> str:
    """
    Heurística simple: agarra la palabra después de 'en' o 'in'.
    Si no encuentra nada, devuelve todo el texto (fallback).
    """
    # Intentamos capturar "en Montevideo", "in Montevideo", etc.
    m = re.search(r"(?:en|in)\s+([A-Za-zÁÉÍÓÚáéíóúñÑ]+)", text)
    if m:
        return m.group(1)
    return text


def call_weather_tool_from_text(text: str) -> str:
    """Extrae ciudad y llama a @tool weather_tool(city: str)."""
    city = extract_city_from_text(text)
    # Por seguridad usamos invoke con dict de argumentos
    result = weather_tool.invoke({"city": city})
    return result


def call_db_knowledge(query: str) -> str:
    """Llama a la tool db_knowledge(query: str) sobre Pinecone."""
    result = db_knowledge.invoke({"query": query})
    return result


# -----------------------
# Nodos del grafo
# -----------------------

def weather_tool_node(state: MessagesState):
    user_text = state["messages"][-1].content

    tool_output = call_weather_tool_from_text(user_text)

    tool_msg = AIMessage(
        content=f"[weather_tool]\n{tool_output}",
        name="weather_tool"
    )
    return {"messages": state["messages"] + [tool_msg]}


def wikipedia_tool_node(state: MessagesState):
    user_text = state["messages"][-1].content

    tool_output = call_wikipedia_tool(user_text)

    tool_msg = AIMessage(
        content=f"[wikipedia_tool]\n{tool_output}",
        name="wikipedia_tool"
    )
    return {"messages": state["messages"] + [tool_msg]}


def db_knowledge_node(state: MessagesState):
    user_text = state["messages"][-1].content

    tool_output = call_db_knowledge(user_text)

    tool_msg = AIMessage(
        content=f"[db_knowledge]\n{tool_output}",
        name="db_knowledge"
    )
    return {"messages": state["messages"] + [tool_msg]}


from langchain_core.messages import SystemMessage, AIMessage, HumanMessage

def chatbot_node(state: MessagesState):
    """
    Nodo final que llama al LLM.
    - Identifica qué tool se usó (si alguna).
    - Instruye al modelo para que use su salida y no diga que no tiene acceso.
    - La respuesta del modelo SIEMPRE empieza con 'TOOL USADA: ...'.
    """
    messages = state["messages"]

    # 1) Detectar la última tool usada (si la hay)
    tool_used = "ninguna"
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and msg.name is not None:
            tool_used = msg.name
            break

    # 2) System prompt fuerte para el modelo
    system_text = f"""
Eres un asistente que responde al usuario basándote en el historial de mensajes.

- Si ves mensajes de herramientas (AIMessage con name 'weather_tool', 'wikipedia_tool' o 'db_knowledge'),
  DEBES usar su contenido como fuente principal de tu respuesta.
- Si la herramienta 'weather_tool' fue utilizada, NUNCA digas que no tienes acceso a información en tiempo real;
  simplemente confía en el resultado que la herramienta entregó.
- Si la herramienta 'wikipedia_tool' fue utilizada, responde usando el texto de Wikipedia como base.
- Si la herramienta 'db_knowledge' fue utilizada, responde usando como base el contenido devuelto del índice Pinecone.
- Si no se usó ninguna herramienta, responde normalmente solo con tu conocimiento.

IMPORTANTE:
- Tu respuesta SIEMPRE debe comenzar con una línea exactamente así:
  TOOL USADA: {tool_used}
  (si no se usó herramienta, debes poner 'TOOL USADA: ninguna')
- Después de esa línea, escribe la explicación o respuesta al usuario.
"""

    full_messages = [SystemMessage(content=system_text)] + messages

    # 3) Llamar al modelo
    ai_resp = llm.invoke(full_messages)

    # No ponemos name al chatbot, para poder distinguir tool vs. respuesta final
    return {"messages": state["messages"] + [ai_resp]}


# -----------------------
# Router (condiciones)
# -----------------------

def tool_selector(state: MessagesState):
    last_message = state["messages"][-1]

    if not isinstance(last_message, HumanMessage):
        return "chatbot"

    content = last_message.content.lower()

    if "clima" in content:
        return "weather_tool_node"
    if "wiki" in content:
        return "wikipedia_tool_node"
    if "documento" in content:
        return "db_knowledge_node"

    # Cualquier otro mensaje → directo al chatbot
    return "chatbot"


# -----------------------
# Construcción del grafo
# -----------------------

graph_builder = StateGraph(MessagesState)

# Registrar nodos
graph_builder.add_node("weather_tool_node", weather_tool_node)
graph_builder.add_node("wikipedia_tool_node", wikipedia_tool_node)
graph_builder.add_node("db_knowledge_node", db_knowledge_node)
graph_builder.add_node("chatbot", chatbot_node)

# Desde START elegimos a qué nodo ir
graph_builder.add_conditional_edges(
    START,
    tool_selector,
    ["weather_tool_node", "wikipedia_tool_node", "db_knowledge_node", "chatbot"],
)

# Si pasamos por una tool, luego vamos al chatbot
graph_builder.add_edge("weather_tool_node", "chatbot")
graph_builder.add_edge("wikipedia_tool_node", "chatbot")
graph_builder.add_edge("db_knowledge_node", "chatbot")

# El chatbot siempre cierra el flujo
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))


### Interactuar con el modelo

Ejecuta las siguientes pruebas para validar el funcionamiento del agente con diferentes tipos de consultas.

In [ ]:
from langchain_core.messages import HumanMessage

print("=== CLIMA ===")
res_clima = graph.invoke({"messages": [HumanMessage(content="¿Cómo está el clima hoy en Montevideo?")]})
for m in res_clima["messages"]:
    print(type(m).__name__, "->", getattr(m, "name", None), "\n", m.content, "\n")

print("=== WIKI ===")
res_wiki = graph.invoke({"messages": [HumanMessage(content="Buscá en la wiki información sobre la inteligencia artificial.")]})
for m in res_wiki["messages"]:
    print(type(m).__name__, "->", getattr(m, "name", None), "\n", m.content, "\n")

print("=== DOCUMENTO ===")
res_doc = graph.invoke({"messages": [HumanMessage(content="Explicame el contenido del documento que cargamos.")]})
for m in res_doc["messages"]:
    print(type(m).__name__, "->", getattr(m, "name", None), "\n", m.content, "\n")

print("=== CHAT DIRECTO ===")
res_chat = graph.invoke({"messages": [HumanMessage(content="Contame un chiste corto.")]})
for m in res_chat["messages"]:
    print(type(m).__name__, "->", getattr(m, "name", None), "\n", m.content, "\n")
